In [1]:
import gurobipy as gp
import pandas as pd
import geopandas as gpd
from gurobipy import Model, GRB, quicksum
import numpy as np
import pickle
import os

In [2]:
from typing import Dict, Any

In [3]:
mun_origem_df = pd.read_csv('Downloads/CD_MUN_origem.csv')
mun_alvo_df = pd.read_csv('Downloads/CD_MUN_alvo.csv')

In [4]:
centros_df = pd.read_csv('Downloads/cnes_cnv_estabse_115.csv', sep=';')

# OPEN ROUTES SERVICES
rotas_df = pd.read_csv('Downloads/rotas_75x25_FONTES_DESTINOS.csv')

In [5]:
# Conjuntos
I = mun_origem_df['CD_MUN'].tolist() # Municípios
J = mun_alvo_df['CD_MUN'].tolist() # Municípios candidatos
E = ['Ortopedia','Proctologia','Urologia'] # Especialidades médicas

In [6]:
# DOS PARÂMETROS/SUS PARA OFERECIMENTO DAS ESPECIALIDADES
df_demanda = pd.read_csv('Downloads/SE_demandas.csv')

# Define a coluna de municípios como índice
df_demanda.set_index('CD_MUN', inplace=True)

# 1. Renomear colunas para o formato 'Especialidade' (sem o prefixo 'Demanda_')
# O nome da especialidade deve corresponder aos seus 'sets' no Gurobi (E)
df_demanda.columns = ['Ortopedia', 'Proctologia', 'Urologia']

# 2. Converter para dicionário de dicionários (para HE_ie)
# O formato final será: {'Mun1': {'Ortopedia': 5.2, 'Proctologia': 1.5, ...}, ...}
HE_dict = df_demanda.to_dict(orient='index')

In [7]:
# 1. Parâmetros de Produtividade (Consultas/FTE/ano)
# Calculado com base no Quadro 35 da Portaria 1631: (Consultas/100K) / (Médicos/100K)
PRODUCTIVITY_MAP: Dict[str, float] = {
    'Ortopedia': 1500.0,      # 15.000 / 10 = 1.500
    'Proctologia': 800.0,       # 1.600 / 2.0 = 800
    'Urologia': 1166.67,      # 3.500 / 3.0 ≈ 1166.67
}
    
def calculate_ke_ie_dict(file_path: str) -> Dict[int, Dict[str, float]]:
    """
    Carrega os dados de produção, calcula a oferta em FTE (KE_ie) por município e especialidade,
    e retorna o resultado no formato de dicionário de dicionários.

    Args:
        file_path: Caminho para o arquivo CSV contendo a produção de consultas.

    Returns:
        Um dicionário no formato {codigo_mun: {especialidade: KE_ie_FTE}}.
    """
    df = pd.read_csv(file_path)
    
    # Identificar as colunas de código do município e de quantidades
    mun_col = 'CD_MUN'
    qty_cols = [col for col in df.columns if 'Qtd.aprovada' in col]
    
    # Renomear as colunas de quantidade para os nomes das especialidades
    # Ex: 'Ortopedia_Qtd.aprovada' -> 'Ortopedia'
    col_mapping = {col: col.split('_')[0] for col in qty_cols}
    df = df.rename(columns=col_mapping)
    
    # 2. Preparação dos Dados
    # Selecionar apenas o código do município e as colunas de especialidade renomeadas
    specialty_cols = list(col_mapping.values())
    df_prod_wide = df[[mun_col] + specialty_cols].copy()

    # Converter o formato wide (código do município e várias colunas de especialidade)
    # para o formato long (código do município, uma coluna de especialidade, uma coluna de quantidade)
    df_prod_long = df_prod_wide.melt(
        id_vars=mun_col,
        var_name='Especialidade',
        value_name='Producao_Anual_Consultas'
    )
    
    # 3. Cálculo do KE_ie (Oferta Existente em FTE)
    
    def apply_ke_ie_calculation(row: pd.Series) -> float:
        """Aplica a fórmula KE_ie = Produção Anual / Produtividade Anual."""
        especialidade = row['Especialidade']
        producao_anual = row['Producao_Anual_Consultas']
        
        # 3.b Obter a Produtividade
        if especialidade in PRODUCTIVITY_MAP:
            productivity = PRODUCTIVITY_MAP[especialidade]
            
            # 3.c Calcular KE_ie em FTE
            ke_ie = producao_anual / productivity
            return round(ke_ie, 4)  # Arredondar para 4 casas decimais para precisão
        
        return 0.0

    df_prod_long['KE_ie_FTE'] = df_prod_long.apply(apply_ke_ie_calculation, axis=1)

    # 4. Formatar o resultado em dicionário de dicionários
    # Pivotar o DataFrame de volta para o formato wide, com municípios como índice
    ke_ie_df = df_prod_long.pivot(
        index=mun_col, 
        columns='Especialidade', 
        values='KE_ie_FTE'
    )
    
    # Converter o DataFrame pivotado para o dicionário final
    ke_ie_dict = ke_ie_df.to_dict(orient='index')
    
    return ke_ie_dict

# Executar a função com o seu arquivo
file_path = 'Downloads/SE_mun_producao_consultas.csv'
KE_dict = calculate_ke_ie_dict(file_path)

print("--- Dicionário KE_ie (Oferta Existente em FTE) ---")
print(KE_dict)

--- Dicionário KE_ie (Oferta Existente em FTE) ---
{2800100: {'Ortopedia': 0.0253, 'Proctologia': 0.0088, 'Urologia': 0.0111}, 2800209: {'Ortopedia': 0.2113, 'Proctologia': 0.0537, 'Urologia': 0.1234}, 2800308: {'Ortopedia': 24.902, 'Proctologia': 7.7988, 'Urologia': 7.7545}, 2800407: {'Ortopedia': 0.1447, 'Proctologia': 0.1163, 'Urologia': 0.084}, 2800506: {'Ortopedia': 0.2127, 'Proctologia': 0.0975, 'Urologia': 0.1037}, 2800605: {'Ortopedia': 0.9233, 'Proctologia': 0.1938, 'Urologia': 0.1869}, 2800670: {'Ortopedia': 0.242, 'Proctologia': 0.1338, 'Urologia': 0.1526}, 2800704: {'Ortopedia': 0.0473, 'Proctologia': 0.0037, 'Urologia': 0.0197}, 2801009: {'Ortopedia': 0.1753, 'Proctologia': 0.0775, 'Urologia': 0.2537}, 2801108: {'Ortopedia': 0.0673, 'Proctologia': 0.0013, 'Urologia': 0.0129}, 2801207: {'Ortopedia': 0.2433, 'Proctologia': 0.0688, 'Urologia': 0.1226}, 2801306: {'Ortopedia': 0.2847, 'Proctologia': 0.0875, 'Urologia': 0.1483}, 2801405: {'Ortopedia': 0.17, 'Proctologia': 0.1087

In [8]:
# Parâmetros - Dados de demografia e infraestrutura
P_j = dict(zip(
    centros_df['CD_MUN'],
    (centros_df['Total'] > 0).astype(int)
))        # 1 se j é um município provedor de serviços com capacidade (fixo)

In [9]:
# Parâmetros - Mínimos
F_min = 2           # nº mínimo de centros de saúde

r_km = 1.00     # R$/km (custo por km rodado) [ARTIGO ajustado]
s_kmh = 60.0     # km/h (velocidade média) [ARTIGO]
h = 5.0  # tempo na fila e de atendimento [ARTIGO]
m_renda = 7.44 # renda média por hora do cidadão sergipano [IBGE]
F_visitas = 2000.0         # visitas/ano (frequência) [ARTIGO]

In [10]:
## Parâmetros - Custos e outros
CI = 1_800_000 + 1.5*207_365          # custo anual médio do centro - média de 5.14 profissionais por centro
S_e = {
    'Ortopedia': 87466.2,  # Exemplo em R$
    'Proctologia': 88902.84, # Exemplo em R$
    'Urologia': 91976.88   # Exemplo em R$
}

# Deslocamento máximo de um paciente para a especialidade e
DM_por_especialidade = {
    'Ortopedia': 180.0,
    'Proctologia': 180.0,
    'Urologia': 180.0
}

# Dicionários de Parâmetros
D_ij = {}              # Distância entre (i, j)
CD_ij = {}             # Custo de Deslocamento (sem alpha) entre (i, j)
K_rotas_validas = set()  # Conjunto final de pares (i, j) válidos, K
alpha_CD_eij = {}      # O termo final da FO: alpha_eij * CD_ij

# 1. Iterar sobre as rotas e calcular custos e acessibilidade
for _, row in rotas_df.iterrows():
    i = row['COD_ORIGEM']
    j = row['COD_DESTINO']
    dist_km = row['DISTANCIA_KM']
    
    # Ignorar rotas inválidas ou com distância zero (a menos que i=j)
    if dist_km <= 0 and i != j:
        continue
        
    # 1.a. CÁLCULO DE CD_ij (Custo Social Anual por Paciente)
    # CD_ij = [2*r*D_ij + (2*(D_ij/s)+h)*m] * t
    custo_anual_por_paciente = ((
        (2 * r_km * dist_km) + 
        (2 * (dist_km / s_kmh) + h) * m_renda)
    ) * F_visitas

    CD_ij[(i, j)] = custo_anual_por_paciente
    
    # 2.b. CÁLCULO DE alpha_eij * CD_ij (Termo Final da FO)
    for e, DM_e in DM_por_especialidade.items():
        
        # Se a distância for aceitável para qualquer especialidade, o par (i, j) é um candidato de rota.
        if dist_km <= DM_e:
            K_rotas_validas.add((int(i), int(j)))
        
        # Cálculo do Alpha_eij (Fator de Decaimento)
        if dist_km > DM_e:
            alpha_e = 0.0
        else:
            alpha_e = max(DM_e - dist_km, 0) / DM_e
            
        # Termo final de custo social a ser minimizado na FO
        alpha_CD_eij[(e, i, j)] = alpha_e * custo_anual_por_paciente

# 3. Tratamento dos Pares Diagonais (i = j)
# Garante que municípios candidatos (j) possam se auto-atender (se i=j e i está em J)
for i in I:
    if i in J:
        # Se o município de origem for o mesmo do destino:
        if (i, i) not in K_rotas_validas:
            K_rotas_validas.add((int(i), int(i)))
            CD_ij[(i, i)] = 0.0
        
        for e in DM_por_especialidade.keys():
            # Alpha para i=j é sempre 1.0 (máxima cobertura/menor custo de decaimento)
            alpha_CD_eij[(e, i, i)] = 0.0 # Custo de deslocamento é zero se i=j

In [11]:
# Modelo (Gurobi)
m = gp.Model("Localizacao_Centros_Medicos")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2732058
Academic license 2732058 - for non-commercial use only - registered to na___@usp.br


In [12]:
# Variáveis (Gurobi)
x = m.addVars(E, K_rotas_validas, vtype=GRB.BINARY, name="x")
z = m.addVars(J, vtype=GRB.BINARY, name="z")
w = m.addVars(E, J, vtype=GRB.INTEGER, name="w")
m.update()

In [13]:
# Função Objetivo
custo_centros = gp.quicksum(CI * z[j] for j in J)
custo_especialistas = gp.quicksum(S_e[e] * w[e, j] for e in E for j in J)
custo_pacientes = gp.quicksum(
        alpha_CD_eij[(e, i, j)] * HE_dict[i][e] * x[e, i, j]
        for e in E for i, j in K_rotas_validas
        if i in HE_dict and e in HE_dict[i] # Garantir que o HE_ie existe
    )

m.setObjective(custo_centros + custo_especialistas + custo_pacientes, GRB.MINIMIZE)

In [14]:
# Restrições
## R1
for e in E:
        for i in I:
            rotas_i_validas = [(int(origem), int(destino)) for origem, destino in K_rotas_validas if origem == i]
        if rotas_i_validas:
            m.addConstr(
                gp.quicksum(x[e, origem, destino] for origem, destino in rotas_i_validas) == 1,
                name=f"R1_Demanda_Servico_Satisfeita_{e}_{i}"
            )

## R3
for e in E:
        for i, j in K_rotas_validas:
            m.addConstr(x[e, i, j] <= z[j], name=f"R3_Atribuicao_Especialidade_{e}_{i}_{j}")
        
## R5
for e in E:
        for j in J:
            if (j, j) in K_rotas_validas:
                m.addConstr(x[e, j, j] == z[j], name=f"R5_Atendimento_Local_Obrigatorio_{e}_{j}")

In [15]:
# Restrição - R8
m.addConstr(gp.quicksum(z[j] for j in J) >= F_min, name="R8_Min_Centros_Medicos")

<gurobi.Constr *Awaiting Model Update*>

In [16]:
# Restrição - R10
for j in J:
    if P_j[j] == 1:
            m.addConstr(z[j] >= P_j[j], name=f"R10_Forcar_Infraestrutura_Existente_{j}")

In [17]:
# R12
for e in E:
    for j in J:
        # Soma das demandas potenciais de cada i alocadas a j
        demanda_ajustada_j = gp.quicksum(
            max(HE_dict.get(i_orig, {}).get(e, 0.0) - KE_dict.get(i_orig, {}).get(e, 0.0), 0.0)
            * x[e, i_orig, j]
            for i_orig, j_dest in K_rotas_validas
            if j_dest == j
        )

        # Restrição: w_ej >= ∑_i max(HE_ie - KE_ie, 0) * x_eij
        m.addConstr(
            w[e, j] >= demanda_ajustada_j,
            name=f"R12_Capacidade_Especialistas_{e}_{j}"
        )

In [18]:
print(f"\nModelo redefinido com: {m.numVars} variáveis, {m.numConstrs} restrições.")
m.update()


Modelo redefinido com: 5101 variáveis, 0 restrições.


In [19]:
# Otimização
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2732058 - for non-commercial use only - registered to na___@usp.br
Optimize a model with 5179 rows, 5101 columns and 15337 nonzeros
Model fingerprint: 0x2e51fb07
Variable types: 0 continuous, 5101 integer (5026 binary)
Coefficient statistics:
  Matrix range     [3e-02, 4e+01]
  Objective range  [2e+02, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 7.167352e+07
Presolve removed 5179 rows and 5101 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 7.16735e+07 7.16735e+07 

Optimal solution found (tolerance 1.0

In [26]:
# Extração e Análise de Resultados
# if m.status == GRB.OPTIMAL:
if m.status:
    print(f"Custo Total Anual Ótimo (Governo + Social): R${m.objVal:,.2f}")
    
    # Custo do Governo
    custo_governo = custo_centros.getValue()+custo_especialistas.getValue()
    print(f"\nCusto Anual do Governo: R${custo_governo:,.2f}")
    
    # Custo Social
    custo_social = custo_pacientes.getValue()
    print(f"Custo Social de Deslocamento: R${custo_social:,.2f}")
    
    # -----------------------------------------------------
    # Análise das Decisões de Localização (Variável z_j)
    # -----------------------------------------------------
    
    centros_abertos = [j for j in J if z[j].X > 0.5]
    novos_centros = [j for j in J if z[j].X > 0.5 and P_j[j] == 0]
    centros_existentes = [j for j in J if z[j].X > 0.5 and P_j[j] == 1]
    
    print(f"\nTotal de Centros Médicos (CEMs) a serem operados: {len(centros_abertos)}")
    if novos_centros:
        print(f"- Municípios Selecionados: {novos_centros}")
    else:
        print("Nenhum novo Centro Médico foi alocado na solução ótima (pode ter atingido o limite F ou o custo é muito alto).")
    print(f"- Centros Existentes (CEAE/CEM) a serem mantidos/expandidos: {len(centros_existentes)}")
    print(f"- Novos Centros Médicos (CEMs) a serem instalados: {len(novos_centros)}")

    # -----------------------------------------------------
    # Análise das Alocações (Variável we_ej)
    # -----------------------------------------------------
            
    # Número total de especialistas a serem contratados
    alocacao_especialistas = {j: {} for j in J}
    total_especialistas_contratados = 0
    
    
    for j in centros_abertos:
        num_especialistas = 0
        for e in E:
            if (e, j) in w:  # Check if the key exists
                num_especialistas += int(round(w[e, j].X))  # Sum specialists for this specialty
        if num_especialistas > 0:
            alocacao_especialistas[j] = num_especialistas
            total_especialistas_contratados += num_especialistas

    if total_especialistas_contratados > 0:
        print(f"Total de Especialistas Contratados: {total_especialistas_contratados}")
        print("\nDetalhes da Alocação:")
        
        centros_sem_contratacao = [j for j in centros_abertos if not alocacao_especialistas[j]]
        if centros_sem_contratacao:
            print("\nCentros abertos que não exigiram contratação de novos especialistas (capacidade local suficiente):")
            print(f" - {centros_sem_contratacao}")
        else:
            print("Nenhum centro não exigiu contratação de novos especialistas.")
    else:
        print("Nenhum especialista adicional foi contratado na rede. A capacidade existente (K_i) pode ser suficiente.")
            
else:
    print(f"O modelo não encontrou uma solução ótima. Status: {m.status}")
if m.status == GRB.INFEASIBLE:
      print("O modelo é inviável. Verifique as restrições de nível de serviço (F, Pq) ou os limites de capacidade.")

Custo Total Anual Ótimo (Governo + Social): R$71,673,519.84

Custo Anual do Governo: R$71,673,519.84
Custo Social de Deslocamento: R$0.00

Total de Centros Médicos (CEMs) a serem operados: 24
Nenhum novo Centro Médico foi alocado na solução ótima (pode ter atingido o limite F ou o custo é muito alto).
- Centros Existentes (CEAE/CEM) a serem mantidos/expandidos: 24
- Novos Centros Médicos (CEMs) a serem instalados: 0
Total de Especialistas Contratados: 237

Detalhes da Alocação:
Nenhum centro não exigiu contratação de novos especialistas.
